In [368]:
import pandas as pd
import pickle

from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_colwidth', None)

In [369]:
df_products = pd.read_json(f'../../../data/interim/final/products.json.gz', orient="records", compression="gzip")

In [370]:
indices = pd.Series(df_products.index, index = df_products['product_id'])

In [371]:
df_products.head()

,product_id,name,description,price,image_url
0,B00001W0DG,Sony MDR-V500DJ Monitor Series Headphones with Swivel Earcups (Discontinued by Manufacturer),"Revel in high-quality audio with the MDR-V500DJ Studio Monitor Series DJ headphones. Featuring reversible earcups with remix applications, these comfortable headphones are the ideal DJ companion.",6.61,[https://images-na.ssl-images-amazon.com/images/I/21UDx%2BHFMJL.jpg]
1,B00004TLW2,Fujifilm MX2900 2.3MP Digital Camera w/ 3x Optical Zoom Bundle,"The FujiFilm MX-2900 digital camera includes some of the best features in digital imaging. With 2.3 megapixels packed into a 1/1.7-inch CCD, this camera offers the ability to create high resolution images as large as 1,800 x 1,200 pixels. You can print photo-quality images up to 8 by 10 inches. A primary color filter and auto white balance maintain the realistic images you want to create--grass stays green and the sky remains blue. The optical 3x zoom lens introduces versatility to your image capturing. It delivers coverage equivalent to a 35-105mm range in 35mm cameras. Your close-ups and portraits will have the look and feel of traditional photographs. The camera boots up in no time at all and the interval between frames is short so you won't miss a shot.",112.27,"[https://images-na.ssl-images-amazon.com/images/I/41VAMXNBNPL.jpg, https://images-na.ssl-images-amazon.com/images/I/41QMVYA6RFL.jpg, https://images-na.ssl-images-amazon.com/images/I/41NKGDW17HL.jpg, https://images-na.ssl-images-amazon.com/images/I/318C9AXX78L.jpg]"
2,B00004VUM1,Sony MVC-FD95 Mavica 2MP Digital Camera with 10x Optical Zoom,w/ Canon SELPHY CP760 Compact Photo Printer 32 MB MultiMediaCard SD Memory Card SDHC Memory Card MultiMediaCardplus F/2.8-4.9Product TypeDigital camera - compact Width3.4 in Depth0.9 in Height2.2 in Weight,99.00,[https://images-na.ssl-images-amazon.com/images/I/51ZQT04EX5L.jpg]
3,B00004WFYN,Plantronics H141 Duoset Convertible Headset (Discontinued by Manufacturer),"- Convertible headset<br />- Quick disconnect (PL-M22 or PL-P10 adapter required)<br />- Convertible to over-the-ear or over-the-head (comes over-the-ear, with kit for over-the-head conversion)<br />- Will work on either ear<br />- Earpiece has plum-colored rubber trim for stability<br />- Voicetube microphone<br />- Includes extra voicetube (PL-29960-01)<br />- Black<br /><br />Package Contents:<br />- Earpiece<br />- Cord<br />- Clothing clip<br />- Headband<br />- Foam earpiece cover<br />- Extra voicetube<br /><br />ADDITIONAL/REPLACEMENT PARTS:<br />PL-43298-01: Headband<br />PL-45651-01: Earloop<br />PL-46186-01 Leatherette ear cushion<br />PL-62800-01: Behind-the-head neckband<br /><br />PLH141",59.00,"[https://images-na.ssl-images-amazon.com/images/I/31KN5QDRGGL.jpg, https://images-na.ssl-images-amazon.com/images/I/31B0TQQPP4L.jpg, https://images-na.ssl-images-amazon.com/images/I/311ZR51ZWZL.jpg, https://images-na.ssl-images-amazon.com/images/I/41YMMA6AWBL.jpg, https://images-na.ssl-images-amazon.com/images/I/314RJ58GT8L.jpg, https://images-na.ssl-images-amazon.com/images/I/312KRDJAKDL.jpg, https://images-na.ssl-images-amazon.com/images/I/41PJM3SF33L.jpg]"
4,B00004XSHN,Fujifilm FinePix 4900 4.3MP Digital Camera w/ 6x Optical Zoom,"Fuji's FinePix 4900 is one of a new style of consumer digital cameras appearing on the market. Borrowing features from both film SLRs and point-and-shoot digital cameras, these hybrids offer an extremely potent set of features in a compact package and will undoubtedly steer even more film photographers over to digital.",60.00,"[https://images-na.ssl-images-amazon.com/images/I/51ME06E3MRL.jpg, https://images-na.ssl-images-amazon.com/images/I/51ESWN8RSML.jpg]"


In [372]:
item_similarity_matrix = pickle.load(open(f'../../../models/content_based_filtering/item_similarity_matrix.pkl', 'rb'))

In [373]:
def recommend(product_id, n = 10):    
    if product_id not in indices.index:
        print("Product not found.")
        return
    
    idx = indices[product_id]

    scores = pd.Series(item_similarity_matrix[idx]).sort_values(ascending = False)
    
    top_n_idx = list(scores.iloc[1:n].index)

    return df_products.iloc[top_n_idx]

In [374]:
product_id = 'B00TKNVPLG'

In [375]:
print(f'Querying {product_id}:')
df = df_products[df_products['product_id'] == product_id]
df[['product_id', 'name', 'description']].head()

Querying B00TKNVPLG:


,product_id,name,description
1039,B00TKNVPLG,Samson SR450 Closed Back On-Ear Studio Headphones,"Samson's latest Studio Reference Series headphones offer exceptional sonic clarity and comfort in a durable, lightweight design that's perfect for studio and mobile applications."


In [376]:
recommend(product_id, 10)[['product_id', 'name', 'description']]

,product_id,name,description
0,B00001W0DG,Sony MDR-V500DJ Monitor Series Headphones with Swivel Earcups (Discontinued by Manufacturer),"Revel in high-quality audio with the MDR-V500DJ Studio Monitor Series DJ headphones. Featuring reversible earcups with remix applications, these comfortable headphones are the ideal DJ companion."
942,B00PHQQ6AU,ABCShopUSA Over-Ear [45mm Dynamic Driver] Professional Studio Monitor Headphones [HP3BK],Over-Ear Professional Studio Monitor Headphones
237,B004DOX70O,Sennheiser HD 65 TV Closed Back Dynamic Headphones for TVs,"The HD 65 TV is a closed back, dynamic stereo headphone designed specifically for watching TV and movies. This comfortable, lightweight around the ear headphone delivers high-quality reproduction of TV sound featuring superb dialog clarity. It is equipped with a 17-foot connection cable with integrated volume control for direct connection to virtually any TV. The volume control features independent left and right control, allowing users to balance the sound to individual preferences."
390,B00A0AOVRS,"MartinLogan Mikros 90 Reference On-Ear Headphones, Black","Inspired by the sonic clarity and detail of our legendary electrostatic speakers, the Mikros 90 reference in-ear on-ear headphones deliver the realism, richness, and purity of sound that MartinLogan's discerning customers have come to expect. Obsessive attention to detail, sleek minimalistic design and high value materials are deeply engrained in our design philosophy that we've extended to Mikros headphones. Designed and voiced in house by the same team that has created 30-years of electrostatic speaker innovation, Mikros 90 was carefully crafted with premium materials like genuine leather, light weight aluminum, and solid noise isolating ear pads. Each piece comes together flawlessly to create a uniquely MartinLogan experience."
527,B00DCXWXEI,AKG Pro Audio K712 PRO Over-Ear Open Reference Studio Headphones,"MASTER THE SOUND for precision listening, mixing and mastering The K712 PROs are reference, open, over-ear headphones for precise listening, mixing and mastering. The over-ear design guarantees maximum wearing comfort for fatigue-free mixing and mastering, while providing spacious and airy sound without any compromise. Their precise powerful sound results from improved low-end performance by 3dB. Being hand-crafted in Austria, the K712 PROs represent the high quality and legendary design AKG is known for."
848,B00N26PPS6,Wireless Bluetooth Earpieces/ Headset/ Headphones for BLU BLU Studio 5.0 C HD (Black) + MYNETDEALS Stylus,"Wireless Bluetooth Earpieces/ Headset/ Headphones for BLU BLU Studio 5.0 C HD/ 5C/ iPhone 5/ 4S/ 4/ iPad Air/ BLU Studio 5.0 C HD/ Mini/ iPad 4/ 3/ 2/ iPod touch (5th, 4th gen)/ - Black"
1036,B00THZ3WZY,KEF M500 Hi-Fi On-Ear Headphones - Aluminum/White,"Your KEF M500 on-ear headphones produces audio from a fine-tuned 1.5 inch neodymium driver built to deliver full range of high definition audio, from 20Hz to 20kHz. To drive home all the intricacies and nuances of your audio, the KEF M500 utilizes copper-clad aluminum wire voice coils and properly mounted apertures in the acoustic chamber to optimize clarity, dynamics and bass performance. Whether they're used as studio headphones or just for everyday listening, the M500 delivers.<br /><br />Outside sounding great, the KEF M500 uses a tangle-free flat cable with an integrated in-line microphone that doubles as a 3-button remote. Your remote gives you the ability to play and pause your music, control your audio volume, and switch between your iPhone phone calls and your music. Top all that off with the headphones' stylish look crafted by our audio engineers alongside Studio F.A. Porsche, of Porsche vehicle fame. The result of our collaboration is a design that enhances the M500 listening experience, from strategically positioned tuning plates to meticulously-planned acoustic chambers, all while retaining a sleek new look.<br />"
8,B00005NKXC,Apple 

In [ ]:
k = 5  # Number of top recommendations to consider
precision = 0
recall = 0
for i in range(len(cosine_sim)):
    top_k_indices = cosine_sim[i].argsort()[:-k-1:-1]
    relevant_indices = test_data.iloc[i]['relevant_indices']
    true_positives = set(top_k_indices).intersection(set(relevant_indices))
    precision += len(true_positives) / k
    recall += len(true_positives) / len(relevant_indices)
precision /= len(cosine_sim)
recall /= len(cosine_sim)
print('Precision@5:', precision)
print('Recall@5:', recall)